In [27]:
import os
import sys
import numpy as np
import pandas as pd

def readFASTA(path):
    f = open(path,"rb")
    data = f.readlines()
    f.close
    pool = []
    tmpLabel = ""
    tmpSeq = ""
    flag = 0
    for each in data:
        each = each[:-1]
        if each[0] == ">":
            if flag == 1:
                pool.append([tmpLabel, tmpSeq])
            tmpLabel = each[1:]
            tmpSeq = ""
        else:
            tmpSeq += each
        flag = 1
    pool.append([tmpLabel, tmpSeq])
    return pool

def reverseComplement(seq):
    basePool = {"A":"T","T":"A","C":"G","G":"C"}
    out_seq = ""
    for e in seq:
        out_seq = basePool[e] + out_seq
    return out_seq

def designProbe(seq, probeLength):
    probes = []
    offset = (len(seq) % probeLength) / 2
    probes.append(seq[:offset])
    for i in np.arange(offset, len(seq), probeLength):
        probes.append(seq[i: i + probeLength])
    probes_final = []
    probes_final.append(probes[0] + probes[1])
    for i in range(len(probes) - 4):
        probes_final.append(probes[i + 2])
    probes_final.append(probes[-2] + probes[-1])
    return probes_final

In [31]:
#Path to rRNA sequences. rRNA sequences should be labelled as [SampleID]_16S and [SampleID]_23S in FASTA format.
input_PATH = "./rRNA_sequence.fa"
output_PATH = "./rRNA_probe.tsv"
probe_length = 50

In [32]:
rRNA_pool = readFASTA(input_PATH)
rRNA_pool_rc = [[e[0], reverseComplement(e[1])] for e in rRNA_pool]
rRNA_pool_probes = [[e[0], designProbe(e[1], probe_length)] for e in rRNA_pool_rc]

In [33]:
f = open(output_PATH, "w")
f.writelines(["rRNA_label\tprobe_ID\tprobe_sequence" + os.linesep])
for eachRNA in rRNA_pool_probes:
    rRNA_label = eachRNA[0]
    probe_pool = eachRNA[1]
    for i in range(len(probe_pool)):
        probe_ID = rRNA_label + "_" + str(i)
        f.writelines([rRNA_label + "\t" + probe_ID + "\t" + probe_pool[i] + os.linesep])
f.close()